In [1]:
import numpy as np
import pandas as pd
import cv2, os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split

# Import classifiers and performance metrics
from sklearn import svm, metrics

In [7]:
# Read dataset

birds_dir = os.listdir('../dataset/actual/birds/')
drones_dir = os.listdir('../dataset/actual/drones/')
birds = []
drones = []
for i in range(200):
    birds.append(cv2.resize(cv2.imread("../dataset/actual/birds/" + birds_dir[i], cv2.IMREAD_GRAYSCALE), (200, 200)).flatten())
    drones.append(cv2.resize(cv2.imread("../dataset/actual/drones/" + drones_dir[i], cv2.IMREAD_GRAYSCALE), (200, 200)).flatten())

In [8]:
img_data = np.concatenate((np.array(birds), np.array(drones)), axis=0)
img_target = []
for i in range(400):
    if i<200:
        img_target.append(0)
    else:
        img_target.append(1)
img_target = np.array(img_target)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(img_data, img_target)

In [10]:
# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001, C=9)
#fit to the trainin data
classifier.fit(X_train,y_train)

SVC(C=9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
y_pred = classifier.predict(X_test)

In [12]:
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(y_test, y_pred)))

Classification report for classifier SVC(C=9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        56
           1       0.44      1.00      0.61        44

   micro avg       0.44      0.44      0.44       100
   macro avg       0.22      0.50      0.31       100
weighted avg       0.19      0.44      0.27       100




/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Confusion matrix:
[[ 0 56]
 [ 0 44]]


In [ ]:
import optunity
import optunity.metrics
import sklearn

# score function: twice iterated 10-fold cross-validated accuracy
@optunity.cross_validated(x=img_data, y=img_target, num_folds=10, num_iter=2)
def svm_auc(x_train, y_train, x_test, y_test, logC, logGamma):
    model = sklearn.svm.SVC(C=10 ** logC, gamma=10 ** logGamma).fit(x_train, y_train)
    decision_values = model.decision_function(x_test)
    return optunity.metrics.roc_auc(y_test, decision_values)

# perform tuning
hps, _, _ = optunity.maximize(svm_auc, num_evals=200, logC=[-5, 2], logGamma=[-5, 1])

# train model on the full training set with tuned hyperparameters
optimal_model = sklearn.svm.SVC(C=10 ** hps['logC'], gamma=10 ** hps['logGamma']).fit(img_data, img_target)